<a href="https://colab.research.google.com/github/daewpark/web_scrapping/blob/main/eunpyung_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Package 설치

In [2]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 12.5MB/s 
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,738 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_6

### Pactkage 호출 및 로드

In [3]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920x1080')
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [4]:
import urllib.request
from bs4 import BeautifulSoup
import re

### Selenium WebDriver 구동 및 검색결과 Parsing

In [5]:
url = "https://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001180"
driver.get(url)
time.sleep(1)

In [6]:
# 로드 된 웹페이지에서 검색창의 Name 을 찾아서 검색어 입력
search = driver.find_element_by_name("searTot")
search.send_keys("원아모집") # headless 모드에서 chrome_options.add_argument('--window-size=1920x1080') 옵션이 없으면 에러 발생하므로 옵션을 반드시 넣을 것
search.submit()


In [7]:
# 검색결과 파싱
driver.switch_to.window(driver.window_handles[-1]) #검색결과가 다른 탭에서 열리므로 새롭게 열린 탭으로 이동
search_html = driver.page_source
soup = BeautifulSoup(search_html,'html.parser')
title = soup.find_all(class_='btnS_nw')

### Pandas DataFrame 으로 결과저장

In [29]:
result_list = []

for i in title :   
   line = i.attrs['title'] + "   " + i.attrs['href']
   result_list.append(line)

with open ("result.txt","w",encoding="utf-8") as file :
    for i in result_list :
        file.write (i + '\n')

file.close


<_io.TextIOWrapper name='result.txt' mode='r' encoding='UTF-8'>


### 스크래핑 결과 메일로 보내기

In [9]:
import smtplib;
# from email.mime.base import MIMEBase;
# 메일 메시지를 만드는 모듈 (MIMEBase는 이하 MIMEMultipart, MIMEText, MIMEApplication, MIMEImage, MIMEAudio)의 상위 모듈이다.
# 굳이 선언할 필요없다.

from email.mime.multipart import MIMEMultipart                                  # 메일의 Data 영역의 메시지를 만드는 모듈 (MIMEText, MIMEApplication, MIMEImage, MIMEAudio가 attach되면 바운더리 형식으로 변환)
from email.mime.text import MIMEText                                            # 메일의 본문 내용을 만드는 모듈
from email.mime.application import MIMEApplication                              # 메일의 첨부 파일을 base64 형식으로 변환
from email.mime.image import MIMEImage                                          # 메일의 이미지 파일을 base64 형식으로 변환(Content-ID 생성)
from email.mime.audio import MIMEAudio                                          # 메일의 음악 파일을 base64 형식으로 변환(Content-ID 생성)
import io # 파일 IO

In [42]:
#메일 발송시간을 알리기 위하여 timestamp 라이브러리 호출
import datetime as dt
today = dt.date.today()
today_string = today.strftime("%Y-%m-%d")

# 메일 서버와 통신하기 전에 메시지를 만든다.
message = MIMEMultipart()
message.set_charset("utf-8")
message['From'] = "dwn.park@gmail.com"
message['To'] = "dwn.park@gmail.com"
message['Cc'] = None
message['Bcc'] = None
message['Subject'] = "은평구청 원아모집 공고 알림봇 (" + today_string + ")"
body = "\r\n".join(result_list)
message.attach (MIMEText(body, 'plain'))
print(message)

MIME-Version: 1.0
Content-Type: multipart/mixed; charset="utf-8"; boundary="===============3608656849532287762=="
Content-Transfer-Encoding: base64
From: dwn.park@gmail.com
To: dwn.park@gmail.com
Cc: 
Bcc: 
Subject: =?utf-8?b?7J2A7Y+J6rWs7LKtIOybkOyVhOuqqOynkSDqs7Xqs6Ag7JWM66a867SHICgyMDIxLTAyLTI1KQ==?=

--===============3608656849532287762==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

6rWs66a9IOusvOu5m+y6kOyKrOyWtOumsOydtOynkSDsm5DslYTrqqjsp5Eg7JWI64K07IOI7LC9
7Je06riwICAgaHR0cDovL3d3dy5lcC5nby5rci9DbXNXZWIvdmlld1BhZ2UucmVxP2lkeD1QRzAw
MDAwMDExMzEmYm9hcmRJZD1CTzAwMDAwMDAwODcmQ1AwMDAwMDAwMDAyX0JPMDAwMDAwMDA4N19B
Y3Rpb249Ym9hcmRWaWV3JkNQMDAwMDAwMDAwMl9CTzAwMDAwMDAwODdfVmlld05hbWU9Ym9hcmQv
Qm9hcmRWaWV3JmJvYXJkRGF0YUlkPTIxMTkzMg0K44CO6rWs66a9IOydkeyVlO2WieuzteyWtOum
sOydtOynkeOAj+ybkOyVhOuqqOynkSDslYjrgrTsg4jssL3sl7TquLAgICBodHRwOi8vd3d3LmVw
LmdvLmtyL0Ntc1dlYi92aWV3UGFnZS5yZXE/aWR4PVBHMDAwMDAwMTEzMSZib2FyZElkPUJPMDAw
MDAwMDA4NyZDUDAwMDAwM

In [43]:
# 첨부 파일 추가
#with open("test.xlsx", "rb") as fp :
#    part = MIMEApplication(fp.read(), Name = "test.xlsx") # Name은 메일 수신자에서 설정되는 파일 이름
#    data.attach (part) # Data 영역의 메시지에 바운더리 추가

# 이미지 파일 추가
#with open("capture.png", 'rb') as fp :
#    img = MIMEImage(fp.read(), Name = "capture.png") # Name은 메일 수신자에서 설정되는 파일 이름
#    img.add_header('Content-ID', '<capture>') # 해더에 Content-ID 추가(본문 내용에서 cid로 링크를 걸 수 있다.)
#    data.attach(img) # Data 영역의 메시지에 바운더리 추가


# Html 형식의 본문 내용 (cid로 이미 첨부 파일을 링크했다.)
# msg = MIMEText("Hello Test<br /><img src='cid:capture'>", 'html')

In [46]:
session = smtplib.SMTP('smtp.gmail.com', 587) # 메일 서버와 통신 개시
session.set_debuglevel(True) 
session.ehlo()
session.starttls()
session.login('dwn.park@gmail.com', 'xytbwmxhgsfjvune')

sender = message['From']
receiver = message['To'].split(",")
if message['Cc'] is not None :
    receiver += data['Cc'].split(",") # 참조자 추가
if message['Bcc'] is not None :
    receiver += message['Bcc'].split(",") # 숨은 참조자 추가
session.sendmail(sender, receiver, message.as_string()) # 메일 프로토콜 상 MAIL, RCPT, DATA 순으로 메시지를 보내야 하는데 이걸 sendmail함수에서 자동으로 해준다.
session.quit() # QUIT을 보내고 접속을 종료하고 메일을 보낸다.

send: 'ehlo [172.28.0.2]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.91.10.12]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [34.91.10.12]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [172.28.0.2]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.91.10.12]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [

(221, b'2.0.0 closing connection w8sm2939641edd.39 - gsmtp')